<a href="https://colab.research.google.com/github/rhamdansyahrulm/Store-Sales/blob/main/Store_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><b>Preparing</b></h1>

<h2> Import Library </h2>

In [1]:
from google.colab import drive
import os
import shutil
import zipfile
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error

<h2> Pull Data </h2>

In [2]:
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
source_file = "/content/drive/MyDrive/Project Data/kaggle.json"
destination_file = '/content/kaggle.json'
shutil.copy2(source_file, destination_file)
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
! chmod 600 /content/kaggle.json

In [ ]:
! kaggle competitions download -c store-sales-time-series-forecasting

In [ ]:
zip_file_path = '/content/store-sales-time-series-forecasting.zip'
extract_dir = '/content/store-sales-time-series-forecasting'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("File successfully extracted!")

<h1><b>Dataset Description</b></h1>

**predict sales for the thousands of product families sold at Favorita stores located in Ecuador**. The training data includes dates, store and product information, whether that item was being promoted, as well as the sales numbers. Additional files include supplementary information that may be useful in building your models.

In [ ]:
folder_dataset = '/content/store-sales-time-series-forecasting'
file_names = [f for f in os.listdir(folder_dataset) if os.path.isfile(os.path.join(folder_dataset, f))]

print(file_names)

<h2><b>Train Datasets<b><h2>

<ul>
    <li>Training data, comprising time series of features:</li>
    <ul>
        <li><strong>store_nbr:</strong> Identifies the store at which the products are sold.</li>
        <li><strong>family:</strong> Identifies the type of product sold.</li>
        <li><strong>onpromotion:</strong> Gives the total number of items in a product family that were being promoted at a store at a given date.</li>
    </ul>
    <li><strong>Target sales:</strong> Gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (e.g., 1.5 kg of cheese).</li>
</ul>

In [ ]:
raw_train_dataset = pd.read_csv(os.path.join(folder_dataset,"train.csv"))
raw_train_dataset

In [ ]:
raw_train_dataset.info()

In [ ]:
print("Unique Value\n")
print(f"store_nbr's Column => jumlah : {len(raw_train_dataset['store_nbr'].unique())}")
print("="*80)
print(raw_train_dataset["store_nbr"].unique())

print(f"\nfamily's Column => jumlah : {len(raw_train_dataset['family'].unique())}")
print("="*80)
print(raw_train_dataset["family"].unique())

In [ ]:
raw_train_dataset.groupby(['store_nbr', 'family'])["family"].count()

In [ ]:
print(raw_train_dataset.isnull().sum())

<h2><b>Test Datasets<b><h2>

<ul>
  <li>The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.</li>
  <li>The dates in the test data are for the 15 days after the last date in the training data.</li>

In [ ]:
raw_test_dataset = pd.read_csv(os.path.join(folder_dataset,"test.csv"))
raw_test_dataset

In [ ]:
raw_test_dataset.info()

In [ ]:
print("Unique Value\n")
print(f"store_nbr's Column => jumlah : {len(raw_test_dataset['store_nbr'].unique())}")
print("="*80)
print(raw_test_dataset["store_nbr"].unique())

print(f"\nfamily's Column => jumlah : {len(raw_test_dataset['family'].unique())}")
print("="*80)
print(raw_test_dataset["family"].unique())

In [ ]:
raw_test_dataset.groupby(['store_nbr', 'family'])["family"].count()

In [ ]:
print(raw_test_dataset.isnull().sum())

<h2><b>stores Dataset<b><h2>

<ul>
  <li>Store metadata, including city, state, type, and cluster.</li>
  <li>cluster is a grouping of similar stores.</li>

In [ ]:
stores = pd.read_csv(os.path.join(folder_dataset,"stores.csv"))
stores.head()

In [ ]:
stores.info()

In [ ]:
print("Unique Value\n")
print(f"City's Column => jumlah : {len(stores['city'].unique())}")
print("="*80)
print(stores["city"].unique())

print(f"\nState's Column => jumlah : {len(stores['state'].unique())}")
print("="*80)
print(stores["state"].unique())

print("Unique Value\n")
print(f"Type's Column => jumlah : {len(stores['type'].unique())}")
print("="*80)
print(stores["type"].unique())

print(f"\nCluster's Column => jumlah : {len(stores['cluster'].unique())}")
print("="*80)
print(stores["cluster"].unique())

<h2><b>Train-Test Datasets<b><h2>

In [ ]:
raw_combined_dataset = pd.concat([raw_train_dataset, raw_test_dataset], ignore_index=True)
raw_combined_dataset

In [ ]:
raw_combined_dataset_stores = raw_combined_dataset.merge(stores, on='store_nbr', how='left')
raw_combined_dataset_stores

In [ ]:
# Memisahkan DataFrame berdasarkan kolom "store_nbr" dan "family"
grouped = raw_combined_dataset_stores.groupby(['store_nbr', 'family'])

# Membuat dictionary untuk menyimpan tabel-tabel yang terpisah
separated_data = {}

# Menyimpan tabel-tabel terpisah dalam dictionary
for group, group_data in grouped:
    key = f"{group[0]}_{group[1]}"
    separated_data[key] = group_data.reset_index(drop=True)

<h2><b>holidays Events<b><h2>

<ul>
  <li>NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
</li>
  <li>Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).
</li>

In [ ]:
holiday = pd.read_csv(os.path.join(folder_dataset,"holidays_events.csv"))
holiday.head()

In [ ]:
holiday.info()

In [ ]:
print("Unique Value\n")
print(f"Type's Column => jumlah : {len(holiday['type'].unique())}")
print("="*80)
print(holiday['type'].unique())

print(f"\nLocale's Column => jumlah : {len(holiday['locale'].unique())}")
print("="*80)
print(holiday['locale'].unique())

print(f"\nLocale Name's Column => jumlah : {len(holiday['locale_name'].unique())}")
print("="*80)
print(holiday['locale_name'].unique())

print(f"\nnDescription's Column => jumlah : {len(holiday['description'].unique())}")
print("="*80)
print(holiday['description'].unique())

In [ ]:
# Fungsi untuk menghapus karakter setelah kata tertentu dari string atau mengembalikan kata tersebut
def process_description(text, words_to_find, words_to_remove):
    text_lower = text.lower()

    for word in words_to_remove:
        desc = re.sub(r'\b' + re.escape(word) + r'\b', '', text_lower)

    for word in words_to_find:
        match = re.search(word, desc)
        if match:
            return word
    return desc

# Kata-kata tempat pemotongan dilakukan
words_to_remove = ['puente', 'recupero', 'traslado']
words_to_find = ['fundacion', 'provincializacion', 'terremoto manabi', 'mundial de futbol brasil', 'fundacion', 'cantonizacion', 'primer dia del ano', 'independencia', 'navidad', 'dia de la madre']

# Mengubah kolom 'description' sesuai dengan proses yang dijelaskan dalam fungsi
holiday_no_transferred = holiday[holiday['transferred'] != True]
holiday_no_transferred = holiday_no_transferred.drop('transferred', axis=1)
holiday_clean = holiday_no_transferred.copy()
holiday_clean['description'] = holiday['description'].apply(lambda x: process_description(x, words_to_find, words_to_remove))

print(f"\nnDescription's Column => jumlah : {len(holiday_clean['description'].unique())}")
print("="*80)
print(holiday_clean['description'].unique())

holiday_clean.head()

In [ ]:
# 1) Mengubah nama kolom "type" menjadi "case"
holiday_clean['date'] = pd.to_datetime(holiday_clean['date'])
holiday_clean.rename(columns={'type': 'case'}, inplace=True)

# 2) Membuat DataFrame baru dengan filter kolom "locale" == "Local", "Regional", dan "National"
holiday_clean_local = holiday_clean[holiday_clean['locale'] == 'Local'].drop(columns=['locale'])
holiday_clean_regional = holiday_clean[holiday_clean['locale'] == 'Regional'].drop(columns=['locale'])
holiday_clean_national = holiday_clean[holiday_clean['locale'] == 'National'].drop(columns=['locale'])

# 3) Pada DataFrame filter "Local", mengubah kolom "locale_name" menjadi "city"
holiday_clean_local.rename(columns={'locale_name': 'city'}, inplace=True)
# Pada DataFrame filter "Regional", mengubah kolom "locale_name" menjadi "state"
holiday_clean_regional.rename(columns={'locale_name': 'state'}, inplace=True)
# Pada DataFrame filter "National", langsung drop kolom "locale_name"
holiday_clean_national.drop(columns=['locale_name'], inplace=True)

In [ ]:
# Cetak DataFrame hasil pemrosesan
print("Local:")
print(holiday_clean_local.head(2))
print()

print("Regional:")
print(holiday_clean_regional.head(2))
print()

print("National:")
print(holiday_clean_national.head(2))

<h2><b>Oil Datasets<b><h2>

<ul>
  <li>Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

In [ ]:
oil = pd.read_csv(os.path.join(folder_dataset,"oil.csv"))
oil

In [ ]:
oil.info()

In [ ]:
# Ubah kolom 'date' menjadi tipe data datetime
oil['date'] = pd.to_datetime(oil['date'])

# Buat rentang tanggal yang diinginkan
date_range = pd.date_range(start=oil['date'].min(), end=oil['date'].max())

# Gabungkan rentang tanggal dengan DataFrame yang ada
missing_dates_oil = pd.DataFrame({'date': date_range})
missing_dates_oil = missing_dates_oil[~missing_dates_oil['date'].isin(oil['date'])]
oil_all_date = pd.concat([oil, missing_dates_oil]).sort_values('date').reset_index(drop=True)

oil_all_date

In [ ]:
# Reshape the features for training and testing
x_train = oil_all_date.loc[~oil_all_date['dcoilwtico'].isnull(), 'date'].values.reshape(-1, 1)
y_train = oil_all_date.loc[~oil_all_date['dcoilwtico'].isnull(), 'dcoilwtico']
x_test = oil_all_date.loc[oil_all_date['dcoilwtico'].isnull(), 'date'].values.reshape(-1, 1)

K = 3
model = KNeighborsRegressor(n_neighbors=3)
model.fit(x_train, y_train)

# Now you can use the model for prediction
predictions = model.predict(x_test)
oil_imputed = oil_all_date.copy()
oil_imputed.loc[oil_imputed['dcoilwtico'].isnull(), 'dcoilwtico'] = predictions

oil_imputed

<h2><b>Merge Datasets<b><h2>

In [ ]:
ready_dataset = {}
encode_columns = ['case_city', 'description_city', 'case_regional', 'description_regional', 'case', 'description']

# Dictionary to store label encoders
label_encoders = {}

for key, value in separated_data.items():
    value['date'] = pd.to_datetime(value['date'])
    merged_df_1 = value.merge(oil_imputed, on='date', how='left')
    merged_df_2 = merged_df_1.merge(holiday_clean_local, on=['date', 'city'], how='left', suffixes=('', '_city'))
    merged_df_3 = merged_df_2.merge(holiday_clean_regional, on=['date', 'state'], how='left', suffixes=('_city', '_regional'))
    merged_df_4 = merged_df_3.merge(holiday_clean_national, on='date', how='left', suffixes=('_regional', ''))
    merged_df_4 = merged_df_4.drop(columns=["city", "state", "store_nbr", "family", "type", "cluster"])
    for encode_column in encode_columns:
        label_encoder = LabelEncoder()
        merged_df_4[encode_column] = label_encoder.fit_transform(merged_df_4[encode_column])
        # Store the label encoder in the dictionary
        label_encoders[encode_column] = label_encoder
    merged_df_4 = merged_df_4.set_index('date')
    merged_df_4.loc[merged_df_4.index[-150:], "sales"] = None
    ready_dataset[key] = merged_df_4

In [ ]:
label_encoders

In [ ]:
print(label_encoders['case'].classes_)
print()
print(label_encoders['description'].classes_)

<h2><b>Create Machine Learning Model<b><h2>

In [ ]:
x_trains = {}
y_trains = {}
x_tests = {}
y_tests = {}
x_predicts = {}

for key, value in ready_dataset.items():
  train = value[~value["sales"].isnull()]
  test = train.loc[train.index[-150:]]
  predict = value[value["sales"].isnull()]

  x_trains[key], y_trains[key] = train.iloc[:,2:], train.iloc[:,1].to_frame()
  x_tests[key], y_tests[key] = test.iloc[:,2:], test.iloc[:,1].to_frame()
  x_predicts[key] = train.iloc[:,2:]

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
y_trains["20_CLEANING"].plot(ax=ax, label='Training Set', title='Data Train/Test Split')
y_tests["20_CLEANING"].plot(ax=ax, label='Test Set')
ax.axvline('11-04-2016', color='black', ls='--')
ax.legend (['Training Set', 'Test Set'])
plt.show()

In [ ]:
a = "10_Rhamdan"
int(a.split("_")[0])

In [ ]:
reg_models = {}

for key, value in x_trains.items():
    while int(key.split("_")[0]) <= 5:
        reg = xgb.XGBRegressor(n_estimators=100, early_stopping_rounds=50)
        reg.fit(x_trains[key], y_trains[key],
                eval_set=[(x_trains[key], y_trains[key]), (x_tests[key], y_tests[key])],
                verbose=False)

        # Store the trained regressor in the dictionary
        reg_models[key] = reg

In [ ]:
reg_models.keys()

In [ ]:
fi = pd.DataFrame (data=reg.feature_importances_,
              index=reg.feature_names_in_,
              columns=['importance'])
fi.sort_values('importance').plot(kind='barh', title='Feature Importance')
plt.show()

In [ ]:
test_model = reg.predict(x_tests["20_CLEANING"])
compare_y = y_tests["20_CLEANING"].copy()
compare_y["predict"] = test_model
compare_y

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
compare_y["sales"].plot(ax=ax, label='Test Set')
compare_y["predict"].plot(ax=ax, label='Test Set')
ax.axvline('03-19-2017', color='black', ls='--')
ax.legend (['Training Set', 'Test Set'])
plt.show()

## Github

In [4]:
!git config --global user.email "rhamdansyahrulm@gmail.com"
!git config --global user.name "rhamdansyahrulm"

In [5]:
!git clone https://github.com/rhamdansyahrulm/Store-Sales.git

Cloning into 'Store-Sales'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.


In [7]:
!git add store_sales.ipynb

fatal: not a git repository (or any of the parent directories): .git
